In [1]:
import json
import pandas as pd
import numpy as np
from ipywidgets import widgets
from numpy.linalg import norm
from neo4j import GraphDatabase

In [ ]:
with open('config.json', 'r') as f:
    credentials = json.load(f)

uri = credentials['uri']
user = credentials['user']
password = credentials['password']
driver = GraphDatabase.driver(uri, auth=(user, password))
#session = driver.session()

In [ ]:
def get_options(driver):
    session = driver.session()
    result = []
    query = """
            MATCH (u:User)-[r]-(n:Symptom)
            RETURN DISTINCT n.symptom AS symptom, COUNT(u) AS ocurrency
            ORDER BY ocurrency DESC
        """
    with session as s:
        tx = s.begin_transaction()
        for row in tx.run(query):
            try:
                if row:
                    symptom = (row[0]).strip()
                    ocurrency = row[1]
                    node_dict = {"symptom": symptom, "ocurrency": ocurrency}
                    result.append(node_dict)
            except Exception as e:
                print(e, row)
        tx.commit()
    return result

In [ ]:
options = get_options()

In [ ]:
widget_seleccion = widgets.SelectMultiple(
    options=opciones,
    value=[opciones[0]],
    description='Selecciona opciones:'
)

display(widget_seleccion)
